# DMIA 2016

## Industry

## Zakharkin Ilya (MIPT Student, 2 course)

### Homework

### Task 0: Рекомендательная система (самые просматриваемые и самые покупаемые)

In [2]:
from collections import OrderedDict
from collections import Counter
import numpy as np
import pandas
import heapq

In [4]:
visit_popularity = Counter()
purchase_popularity = Counter()

with open('../train.txt', 'r') as file:
    for line in file:
        visits, purchases = line.strip().split(';')
        visits = visits.split(',')
        if purchases != '':
            purchases = purchases.split(',')
        visit_popularity.update(visits)
        purchase_popularity.update(purchases)

In [5]:
def recommend_by_visits(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items), 
                           key=lambda x: visit_popularity.get(x, 0))
    
def recommend_by_purchases(items, max_count):
    return heapq.nlargest(max_count, OrderedDict.fromkeys(items),
                           key=lambda x: purchase_popularity.get(x, 0))

In [63]:
def calculate_metrics(recommendation_system, max_count, sessions_file):
    with open(sessions_file, 'r') as file:
        avg_precision = np.zeros(max_count)
        avg_recall = np.zeros(max_count)
        sessions_count = 0
        for line in file:
            visits, purchases = line.strip().split(';')
            if purchases == '':
                continue
            visits = visits.split(',')
            purchases = purchases.split(',')
            recommended = recommendation_system(visits, max_count)
            true_recommended = np.array([])
            for index in range(len(recommended)):
                if recommended[index] in purchases:
                    true_recommended = np.append(true_recommended, recommended[index])
                    avg_precision[index] += len(true_recommended) / (index + 1)
                    avg_recall[index] += len(true_recommended) / len(purchases)
            sessions_count += 1

        return pandas.DataFrame({
            'k': np.arange(max_count) + 1,
            'avg_recall@k': [round(x, 2) for x in avg_recall / sessions_count],
            'avg_precision@k': [round(x, 2) for x in avg_precision / sessions_count]
           }).set_index('k')

In [64]:
calculate_metrics(recommend_by_visits, 1, '../train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.51,0.44


In [65]:
calculate_metrics(recommend_by_visits, 1, '../test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.48,0.42


In [66]:
calculate_metrics(recommend_by_purchases, 1, '../train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.8,0.69


In [67]:
calculate_metrics(recommend_by_purchases, 1, '../test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.53,0.46


In [68]:
calculate_metrics(recommend_by_visits, 5, '../train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.51,0.44
2,0.15,0.21
3,0.07,0.12
4,0.04,0.08
5,0.02,0.05


In [69]:
calculate_metrics(recommend_by_visits, 5, '../test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.48,0.42
2,0.14,0.20
3,0.07,0.12
4,0.03,0.07
5,0.02,0.05


In [70]:
calculate_metrics(recommend_by_purchases, 5, '../train.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.80,0.69
2,0.20,0.20
3,0.08,0.09
4,0.04,0.04
5,0.02,0.03


In [71]:
calculate_metrics(recommend_by_purchases, 5, '../test.txt')

,avg_precision@k,avg_recall@k
k,,
1,0.53,0.46
2,0.14,0.20
3,0.06,0.11
4,0.03,0.07
5,0.02,0.04
